#Introduction to Financial Python


##Pandas-Resampling and DataFrame


###Introduction
In the last chapter we had a glimpse of Pandas. In this chapter we will learn about resampling methods and the DataFrame object, which is a powerful tool for financial data analysis.



###Fetching Data
Here we use the Quandl API to retrieve data.

In [119]:
pip install Quandl

In [120]:
import quandl
import numpy as np
import pandas as pd

In [121]:
quandl.ApiConfig.api_key = 'SzDnauemCyWhqDhQuwTP'

We will create a Series named "aapl" whose values are Apple's daily closing prices, which are of course indexed by dates:

In [122]:
aapl_table = quandl.get('WIKI/AAPL')
aapl = aapl_table['Adj. Close']['2017']
print (aapl)

Date
2017-01-03    114.715378
2017-01-04    114.586983
2017-01-05    115.169696
2017-01-06    116.453639
2017-01-09    117.520300
                 ...    
2017-12-22    175.010000
2017-12-26    170.570000
2017-12-27    170.600000
2017-12-28    171.080000
2017-12-29    169.230000
Name: Adj. Close, Length: 249, dtype: float64


In [123]:
# My example
oilPrice_table = quandl.get("EIA/PET_RWTC_D")
oilprice= oilPrice_table['Value']['2012']
print (oilprice)

Date
2012-01-03    102.96
2012-01-04    103.22
2012-01-05    101.81
2012-01-06    101.56
2012-01-09    101.31
               ...  
2012-12-24     88.29
2012-12-26     90.71
2012-12-27     90.91
2012-12-28     90.66
2012-12-31     91.83
Name: Value, Length: 252, dtype: float64


Recall that we can fetch a specific data point using series['yyyy-mm-dd']. We can also fetch the data in a specific month using series['yyyy-mm'].

In [124]:
print(aapl['2017-3'])

Date
2017-03-01    138.657681
2017-03-02    137.834404
2017-03-03    138.647762
2017-03-06    138.211326
2017-03-07    138.389868
2017-03-08    137.874080
2017-03-09    137.556672
2017-03-10    138.012946
2017-03-13    138.072460
2017-03-14    137.864161
2017-03-15    139.322254
2017-03-16    139.550391
2017-03-17    138.856061
2017-03-20    140.314154
2017-03-21    138.707276
2017-03-22    140.274478
2017-03-23    139.778528
2017-03-24    139.500796
2017-03-27    139.738852
2017-03-28    142.635200
2017-03-29    142.952608
2017-03-30    142.764147
2017-03-31    142.496334
Name: Adj. Close, dtype: float64


In [125]:
# My example
print(oilprice['2012-2'])

Date
2012-02-01     97.63
2012-02-02     96.36
2012-02-03     97.80
2012-02-06     96.89
2012-02-07     98.55
2012-02-08     98.80
2012-02-09     99.88
2012-02-10     98.68
2012-02-13    100.39
2012-02-14    100.82
2012-02-15    101.82
2012-02-16    102.33
2012-02-17    103.27
2012-02-21    105.88
2012-02-22    105.99
2012-02-23    107.44
2012-02-24    109.39
2012-02-27    108.49
2012-02-28    106.59
2012-02-29    107.08
Name: Value, dtype: float64


Or in several consecutive months:

In [126]:
aapl['2017-2':'2017-4']

Date
2017-02-01    127.159749
2017-02-02    126.942467
2017-02-03    127.485673
2017-02-06    128.680728
2017-02-07    129.905412
                 ...    
2017-04-24    142.476496
2017-04-25    143.369205
2017-04-26    142.487208
2017-04-27    142.625281
2017-04-28    142.486415
Name: Adj. Close, Length: 61, dtype: float64

In [127]:
# My example
# Shows the first semester of 2012
oilprice['2012-1':'2012-6']

Date
2012-01-03    102.96
2012-01-04    103.22
2012-01-05    101.81
2012-01-06    101.56
2012-01-09    101.31
               ...  
2012-06-25     78.76
2012-06-26     79.34
2012-06-27     80.27
2012-06-28     77.72
2012-06-29     85.04
Name: Value, Length: 125, dtype: float64

.head(N) and .tail(N) are methods for quickly accessing the first or last N elements.

In [128]:
print(aapl.head(5))
print(aapl.tail(10))

Date
2017-01-03    114.715378
2017-01-04    114.586983
2017-01-05    115.169696
2017-01-06    116.453639
2017-01-09    117.520300
Name: Adj. Close, dtype: float64
Date
2017-12-15    173.87
2017-12-18    176.42
2017-12-19    174.54
2017-12-20    174.35
2017-12-21    175.01
2017-12-22    175.01
2017-12-26    170.57
2017-12-27    170.60
2017-12-28    171.08
2017-12-29    169.23
Name: Adj. Close, dtype: float64


In [129]:
# My example
#First two elements
print(aapl.head(2))
#Last six elements
print(aapl.tail(10))

Date
2017-01-03    114.715378
2017-01-04    114.586983
Name: Adj. Close, dtype: float64
Date
2017-12-15    173.87
2017-12-18    176.42
2017-12-19    174.54
2017-12-20    174.35
2017-12-21    175.01
2017-12-22    175.01
2017-12-26    170.57
2017-12-27    170.60
2017-12-28    171.08
2017-12-29    169.23
Name: Adj. Close, dtype: float64


###Resampling
**series.resample(freq)** is a class called "DatetimeIndexResampler" which groups data in a Series object into regular time intervals. The argument "freq" determines the length of each interval.

**series.resample.mean()** is a complete statement that groups data into intervals, and then compute the mean of each interval. For example, if we want to aggregate the daily data into monthly data by mean:

In [130]:
by_month = aapl.resample('M').mean()
print(by_month)

Date
2017-01-31    118.093136
2017-02-28    132.456268
2017-03-31    139.478802
2017-04-30    141.728436
2017-05-31    151.386305
2017-06-30    147.233064
2017-07-31    147.706190
2017-08-31    158.856375
2017-09-30    157.606500
2017-10-31    157.811627
2017-11-30    172.214500
2017-12-31    171.893100
Freq: M, Name: Adj. Close, dtype: float64


In [131]:
# My example
oil_prices_by_month = oilprice.resample('M').mean()
print(oil_prices_by_month)

Date
2012-01-31    100.273500
2012-02-29    102.204000
2012-03-31    106.157727
2012-04-30    103.321000
2012-05-31     94.654545
2012-06-30     82.303333
2012-07-31     87.895238
2012-08-31     94.131304
2012-09-30     94.513684
2012-10-31     89.491304
2012-11-30     86.531429
2012-12-31     87.859500
Freq: M, Name: Value, dtype: float64


We can also aggregate the data by week:

In [132]:
by_week = aapl.resample('W').mean()
print(by_week.head())

Date
2017-01-08    115.231424
2017-01-15    117.755360
2017-01-22    118.461035
2017-01-29    119.667448
2017-02-05    124.313346
Freq: W-SUN, Name: Adj. Close, dtype: float64


In [133]:
# My example
op_by_week= oilprice.resample('M').mean()
print(op_by_week)

Date
2012-01-31    100.273500
2012-02-29    102.204000
2012-03-31    106.157727
2012-04-30    103.321000
2012-05-31     94.654545
2012-06-30     82.303333
2012-07-31     87.895238
2012-08-31     94.131304
2012-09-30     94.513684
2012-10-31     89.491304
2012-11-30     86.531429
2012-12-31     87.859500
Freq: M, Name: Value, dtype: float64


We can choose almost any frequency by using the format 'nf', where 'n' is an integer and 'f' is M for month, W for week and D for day.



In [134]:
three_day = aapl.resample('3D').mean()
two_week  = aapl.resample('2W').mean()
two_month = aapl.resample('2M').mean()

In [135]:
# My example
op_fifteen_days = oilprice.resample('15D').mean()
op_six_weeks  = oilprice.resample('6W').mean()
op_trimester = oilprice.resample('3M').mean()
print (op_trimester)

Date
2012-01-31    100.273500
2012-04-30    103.967258
2012-07-31     88.383906
2012-10-31     92.601231
2013-01-31     87.179268
Freq: 3M, Name: Value, dtype: float64


Besides the mean() method, other methods can also be used with the resampler:

In [136]:
std = aapl.resample('W').std()    # standard deviation
max = aapl.resample('W').max()    # maximum value
min = aapl.resample('W').min()    # minimum value

In [137]:
aapl.resample('M').max()

Date
2017-01-31    120.443739
2017-02-28    135.999390
2017-03-31    142.952608
2017-04-30    143.597342
2017-05-31    155.469192
2017-06-30    154.821818
2017-07-31    152.839860
2017-08-31    164.000000
2017-09-30    164.050000
2017-10-31    169.040000
2017-11-30    175.880000
2017-12-31    176.420000
Freq: M, Name: Adj. Close, dtype: float64

In [138]:
# My example
op_std = oilprice.resample('W').std()    # standard deviation
op_max = oilprice.resample('W').max()    # maximum value
op_min = oilprice.resample('W').min()    # minimum value

In [139]:
# My example
# the standar deviation of oil prices each month of 2012
oilprice.resample('M').std()

Date
2012-01-31    1.528767
2012-02-29    4.238912
2012-03-31    1.439289
2012-04-30    1.105012
2012-05-31    5.231473
2012-06-30    2.431967
2012-07-31    2.377784
2012-08-31    2.459613
2012-09-30    2.689275
2012-10-31    2.752730
2012-11-30    1.245204
2012-12-31    2.112921
Freq: M, Name: Value, dtype: float64

Often we want to calculate monthly returns of a stock, based on prices on the last day of each month. To fetch those prices, we use the series.resample.agg() method:

In [140]:
last_day = aapl.resample('M').agg(lambda x: x[-1])
print(last_day)

Date
2017-01-31    119.851150
2017-02-28    135.880362
2017-03-31    142.496334
2017-04-30    142.486415
2017-05-31    152.142689
2017-06-30    143.438008
2017-07-31    148.248489
2017-08-31    164.000000
2017-09-30    154.120000
2017-10-31    169.040000
2017-11-30    171.850000
2017-12-31    169.230000
Freq: M, Name: Adj. Close, dtype: float64


In [141]:
# My example
# Last element of each month
op_last_day = oilprice.resample('M').agg(lambda x: x[-1])
print(op_last_day)

Date
2012-01-31     98.46
2012-02-29    107.08
2012-03-31    103.03
2012-04-30    104.89
2012-05-31     86.52
2012-06-30     85.04
2012-07-31     88.08
2012-08-31     96.47
2012-09-30     92.18
2012-10-31     86.23
2012-11-30     88.54
2012-12-31     91.83
Freq: M, Name: Value, dtype: float64


Or directly calculate the monthly rates of return using the data for the first day and the last day:

In [142]:
monthly_return = aapl.resample('M').agg(lambda x: x[-1]/x[1] - 1)
print(monthly_return)

Date
2017-01-31    0.045940
2017-02-28    0.070409
2017-03-31    0.033823
2017-04-30   -0.007736
2017-05-31    0.039829
2017-06-30   -0.073528
2017-07-31    0.033035
2017-08-31    0.047890
2017-09-30   -0.049112
2017-10-31    0.094252
2017-11-30    0.022247
2017-12-31   -0.003357
Freq: M, Name: Adj. Close, dtype: float64


In [143]:
# My example
# returns the monthly return of oil prices for each month of 2012
op_monthly_return = oilprice.resample('M').agg(lambda x: x[-1]/x[1] - 1)
print(op_monthly_return)

Date
2012-01-31   -0.046115
2012-02-29    0.111249
2012-03-31   -0.034214
2012-04-30    0.008364
2012-05-31   -0.177957
2012-06-30    0.012984
2012-07-31    0.003875
2012-08-31    0.106054
2012-09-30   -0.033449
2012-10-31   -0.061493
2012-11-30    0.042874
2012-12-31    0.043049
Freq: M, Name: Value, dtype: float64


Series object also provides us some convenient methods to do some quick calculation.

In [144]:
print(monthly_return.mean())
print(monthly_return.std())
print(monthly_return.max())

0.02114094011940022
0.04775652864223314
0.09425168306576914


In [145]:
# My example
# Information about oil prices monthly returns
print(op_monthly_return.mean())
print(op_monthly_return.std())
print(op_monthly_return.max())

-0.0020650195522376826
0.07826749706854563
0.11124948111249489


Another two methods frequently used on Series are .diff() and .pct_change(). The former calculates the difference between consecutive elements, and the latter calculates the percentage change.

In [146]:
print(last_day.diff())
print(last_day.pct_change())

Date
2017-01-31          NaN
2017-02-28    16.029211
2017-03-31     6.615972
2017-04-30    -0.009919
2017-05-31     9.656274
2017-06-30    -8.704681
2017-07-31     4.810482
2017-08-31    15.751511
2017-09-30    -9.880000
2017-10-31    14.920000
2017-11-30     2.810000
2017-12-31    -2.620000
Freq: M, Name: Adj. Close, dtype: float64
Date
2017-01-31         NaN
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


In [147]:
# My example
# difference an percentage of changes of the prices of oil the last day of each month 
print(op_last_day.diff())
print(op_last_day.pct_change())

Date
2012-01-31      NaN
2012-02-29     8.62
2012-03-31    -4.05
2012-04-30     1.86
2012-05-31   -18.37
2012-06-30    -1.48
2012-07-31     3.04
2012-08-31     8.39
2012-09-30    -4.29
2012-10-31    -5.95
2012-11-30     2.31
2012-12-31     3.29
Freq: M, Name: Value, dtype: float64
Date
2012-01-31         NaN
2012-02-29    0.087548
2012-03-31   -0.037822
2012-04-30    0.018053
2012-05-31   -0.175136
2012-06-30   -0.017106
2012-07-31    0.035748
2012-08-31    0.095254
2012-09-30   -0.044470
2012-10-31   -0.064548
2012-11-30    0.026789
2012-12-31    0.037158
Freq: M, Name: Value, dtype: float64


Notice that we induced a NaN value while calculating percentage changes i.e. returns.

When dealing with NaN values, we usually either removing the data point or fill it with a specific value. Here we fill it with 0:

In [148]:
daily_return = last_day.pct_change()
print(daily_return.fillna(0))

Date
2017-01-31    0.000000
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


In [149]:
# My example
# filling NaN with 0's
op_daily_return = op_last_day.pct_change()
print(op_daily_return.fillna(0))

Date
2012-01-31    0.000000
2012-02-29    0.087548
2012-03-31   -0.037822
2012-04-30    0.018053
2012-05-31   -0.175136
2012-06-30   -0.017106
2012-07-31    0.035748
2012-08-31    0.095254
2012-09-30   -0.044470
2012-10-31   -0.064548
2012-11-30    0.026789
2012-12-31    0.037158
Freq: M, Name: Value, dtype: float64


Alternatively, we can fill a NaN with the next fitted value. This is called 'backward fill', or 'bfill' in short:

In [150]:
daily_return = last_day.pct_change()
print(daily_return.fillna(method = 'bfill'))

Date
2017-01-31    0.133743
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64


In [151]:
# My example
# filling NaN with the next fitted value
op_daily_return = op_last_day.pct_change()
print(op_daily_return.fillna(method = 'bfill'))

Date
2012-01-31    0.087548
2012-02-29    0.087548
2012-03-31   -0.037822
2012-04-30    0.018053
2012-05-31   -0.175136
2012-06-30   -0.017106
2012-07-31    0.035748
2012-08-31    0.095254
2012-09-30   -0.044470
2012-10-31   -0.064548
2012-11-30    0.026789
2012-12-31    0.037158
Freq: M, Name: Value, dtype: float64


As expected, since there is a 'backward fill' method, there must be a 'forward fill' method, or 'ffill' in short. However we can't use it here because the NaN is the first value.

We can also simply remove NaN values by .dropna()

In [152]:
daily_return = last_day.pct_change()
daily_return.dropna()

Date
2017-02-28    0.133743
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067770
2017-06-30   -0.057214
2017-07-31    0.033537
2017-08-31    0.106251
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.016623
2017-12-31   -0.015246
Freq: M, Name: Adj. Close, dtype: float64

In [153]:
# My example
# drop NaN
op_daily_return = op_last_day.pct_change()
op_daily_return.dropna()

Date
2012-02-29    0.087548
2012-03-31   -0.037822
2012-04-30    0.018053
2012-05-31   -0.175136
2012-06-30   -0.017106
2012-07-31    0.035748
2012-08-31    0.095254
2012-09-30   -0.044470
2012-10-31   -0.064548
2012-11-30    0.026789
2012-12-31    0.037158
Freq: M, Name: Value, dtype: float64

###DataFrame
The **DataFrame** is the most commonly used data structure in Pandas. It is essentially a table, just like an Excel spreadsheet.

More precisely, a DataFrame is a collection of Series objects, each of which may contain different data types. A DataFrame can be created from various data types: dictionary, 2-D numpy.ndarray, a Series or another DataFrame.

####Create DataFrames
The most common method of creating a DataFrame is passing a dictionary:

In [154]:
dict = {'AAPL': [143.5, 144.09, 142.73, 144.18, 143.77],'GOOG':[898.7, 911.71, 906.69, 918.59, 926.99],
        'IBM':[155.58, 153.67, 152.36, 152.94, 153.49]}
data_index = pd.date_range('2017-07-03',periods = 5, freq = 'D')
df = pd.DataFrame(dict, index = data_index)
print(df)

              AAPL    GOOG     IBM
2017-07-03  143.50  898.70  155.58
2017-07-04  144.09  911.71  153.67
2017-07-05  142.73  906.69  152.36
2017-07-06  144.18  918.59  152.94
2017-07-07  143.77  926.99  153.49


In [155]:
# My example
my_dict = {'Hours': [1, 7, 5, 33, 2, 5],'Price':[45.56, 34, 65.4, 67.5, 46.6, 56.12],
        'Profits':[10, 14.32, 6.44, 15.33, 23.01, 67.4]}
data_index_2 = pd.date_range('2012-01-01',periods = 6, freq = 'M')
df_2 = pd.DataFrame(my_dict, index = data_index_2)
print(df_2)

            Hours  Price  Profits
2012-01-31      1  45.56    10.00
2012-02-29      7  34.00    14.32
2012-03-31      5  65.40     6.44
2012-04-30     33  67.50    15.33
2012-05-31      2  46.60    23.01
2012-06-30      5  56.12    67.40


####Manipulating DataFrames
We can fetch values in a DataFrame by columns and index. Each column in a DataFrame is essentially a Pandas Series. We can fetch a column by square brackets: df['column_name']

If a column name contains no spaces, then we can also use df.column_name to fetch a column:

In [156]:
df = aapl_table
print (df.Close.tail(5))
print (df['Adj. Volume'].tail(5))

Date
2018-03-21    171.270
2018-03-22    168.845
2018-03-23    164.940
2018-03-26    172.770
2018-03-27    168.340
Name: Close, dtype: float64
Date
2018-03-21    35247358.0
2018-03-22    41051076.0
2018-03-23    40248954.0
2018-03-26    36272617.0
2018-03-27    38962839.0
Name: Adj. Volume, dtype: float64


In [157]:
# My example
msft = quandl.get("wiki/msft")
# Printing last 4 elements of column open
print (msft.Open.tail(4))
# Printing first 6 elements of column low
print (msft['Low'].head(6))


Date
2018-03-22    91.265
2018-03-23    89.500
2018-03-26    90.610
2018-03-27    94.940
Name: Open, dtype: float64
Date
1986-03-13    25.50
1986-03-14    28.00
1986-03-17    29.00
1986-03-18    28.50
1986-03-19    28.00
1986-03-20    27.25
Name: Low, dtype: float64


We may select certain columns of a DataFrame using their names:

In [158]:
aapl_2016 = df['2016']
aapl_month = aapl_2016.resample('M').agg(lambda x: x[-1])
print(aapl_month)

              Open      High     Low  ...    Adj. Low  Adj. Close  Adj. Volume
Date                                  ...                                     
2016-01-31   94.79   97.3400   94.35  ...   91.156128   94.044912   64416504.0
2016-02-29   96.86   98.2300   96.65  ...   93.880927   93.919781   35216277.0
2016-03-31  109.72  109.9000  108.88  ...  105.760531  105.867380   25888449.0
2016-04-30   93.99   94.7200   92.51  ...   89.859540   91.054300   68531478.0
2016-05-31   99.60  100.4000   98.82  ...   96.575559   97.591939   42307212.0
2016-06-30   94.44   95.7700   94.30  ...   92.158220   93.428693   35836356.0
2016-07-31  104.19  104.5500  103.68  ...  101.325177  101.843140   27733688.0
2016-08-31  105.66  106.5699  105.64  ...  103.796505  104.248477   29662406.0
2016-09-30  112.46  113.3700  111.80  ...  109.849008  111.077195   36379106.0
2016-10-31  113.65  114.2300  113.20  ...  111.224577  111.558644   26419398.0
2016-11-30  111.56  112.2000  110.27  ...  108.90800

In [159]:
# My example
# takes rows whose index date is in 2013
msft_2013 = msft['2013']
#rows of last day of every month of 2013
msft_2013_ld = msft_2013.resample('M').agg(lambda x: x[-1])
print(msft_2013_ld)

             Open   High     Low  ...   Adj. Low  Adj. Close  Adj. Volume
Date                              ...                                    
2013-01-31  27.79  27.97  27.400  ...  24.001763   24.045562   50530000.0
2013-02-28  27.88  27.97  27.740  ...  24.498879   24.551868   35840200.0
2013-03-31  28.32  28.66  28.260  ...  24.958122   25.262813   55453800.0
2013-04-30  32.56  33.11  32.280  ...  28.508428   29.232620   75165200.0
2013-05-31  34.82  35.28  34.790  ...  30.935922   31.033736   56165700.0
2013-06-30  34.38  34.79  34.340  ...  30.535773   30.718063   65545500.0
2013-07-31  31.97  32.05  31.710  ...  28.197128   28.312726   43898400.0
2013-08-31  33.37  33.48  33.090  ...  29.634228   29.911853   42790200.0
2013-09-30  33.00  33.31  32.700  ...  29.284957   29.804385   39839500.0
2013-10-31  35.66  35.69  35.340  ...  31.649248   31.707459   41682300.0
2013-11-30  37.82  38.29  37.820  ...  34.128377   34.408117   22090400.0
2013-12-31  37.40  37.58  37.217  ... 

We may select certain columns of a DataFrame using their names:

In [160]:
aapl_bar = aapl_month[['Open', 'High', 'Low', 'Close']]
print(aapl_bar)

              Open      High     Low   Close
Date                                        
2016-01-31   94.79   97.3400   94.35   97.34
2016-02-29   96.86   98.2300   96.65   96.69
2016-03-31  109.72  109.9000  108.88  108.99
2016-04-30   93.99   94.7200   92.51   93.74
2016-05-31   99.60  100.4000   98.82   99.86
2016-06-30   94.44   95.7700   94.30   95.60
2016-07-31  104.19  104.5500  103.68  104.21
2016-08-31  105.66  106.5699  105.64  106.10
2016-09-30  112.46  113.3700  111.80  113.05
2016-10-31  113.65  114.2300  113.20  113.54
2016-11-30  111.56  112.2000  110.27  110.52
2016-12-31  116.65  117.2000  115.43  115.82


In [161]:
# My example
# Only selecting Open and Close columns
msft_bar = msft_2013_ld[['Open','Close']]
print(msft_bar)

             Open   Close
Date                     
2013-01-31  27.79  27.450
2013-02-28  27.88  27.800
2013-03-31  28.32  28.605
2013-04-30  32.56  33.100
2013-05-31  34.82  34.900
2013-06-30  34.38  34.545
2013-07-31  31.97  31.840
2013-08-31  33.37  33.400
2013-09-30  33.00  33.280
2013-10-31  35.66  35.405
2013-11-30  37.82  38.130
2013-12-31  37.40  37.410


We can even specify both rows and columns using loc[]. The row indices and column names are separated by a comma:

In [162]:
print(aapl_month.loc['2016-03':'2016-06',['Open', 'High', 'Low', 'Close']])

              Open    High     Low   Close
Date                                      
2016-03-31  109.72  109.90  108.88  108.99
2016-04-30   93.99   94.72   92.51   93.74
2016-05-31   99.60  100.40   98.82   99.86
2016-06-30   94.44   95.77   94.30   95.60


In [163]:
# My example
# Only selecting Open and Close columns in the first semester of 2013
print( msft_2013_ld.loc['2013-01':'2013-06',['Open','Close']])

             Open   Close
Date                     
2013-01-31  27.79  27.450
2013-02-28  27.88  27.800
2013-03-31  28.32  28.605
2013-04-30  32.56  33.100
2013-05-31  34.82  34.900
2013-06-30  34.38  34.545


The subset methods in DataFrame is quite useful. By writing logical statements in square brackets, we can make customized subsets:

In [164]:
above = aapl_bar[aapl_bar.Close > np.mean(aapl_bar.Close)]
print(above)

              Open      High     Low   Close
Date                                        
2016-03-31  109.72  109.9000  108.88  108.99
2016-08-31  105.66  106.5699  105.64  106.10
2016-09-30  112.46  113.3700  111.80  113.05
2016-10-31  113.65  114.2300  113.20  113.54
2016-11-30  111.56  112.2000  110.27  110.52
2016-12-31  116.65  117.2000  115.43  115.82


In [165]:
# My example
# takes the subset of rows whose open is below the open average
below = msft_bar[msft_bar.Open > np.mean(msft_bar.Open)]
print(below)

             Open   Close
Date                     
2013-05-31  34.82  34.900
2013-06-30  34.38  34.545
2013-08-31  33.37  33.400
2013-09-30  33.00  33.280
2013-10-31  35.66  35.405
2013-11-30  37.82  38.130
2013-12-31  37.40  37.410


####Data Validation
As mentioned, all methods that apply to a Series can also be applied to a DataFrame. Here we add a new column to an existing DataFrame:

In [166]:
aapl_bar['rate_return'] = aapl_bar.Close.pct_change()
print(aapl_bar)

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34          NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [167]:
# My example
# the column rate return in the table will represent the difference between the last day Open of each month
msft_bar['rate_return'] = msft_bar.Open.diff()
print(msft_bar)

             Open   Close  rate_return
Date                                  
2013-01-31  27.79  27.450          NaN
2013-02-28  27.88  27.800         0.09
2013-03-31  28.32  28.605         0.44
2013-04-30  32.56  33.100         4.24
2013-05-31  34.82  34.900         2.26
2013-06-30  34.38  34.545        -0.44
2013-07-31  31.97  31.840        -2.41
2013-08-31  33.37  33.400         1.40
2013-09-30  33.00  33.280        -0.37
2013-10-31  35.66  35.405         2.66
2013-11-30  37.82  38.130         2.16
2013-12-31  37.40  37.410        -0.42


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Here the calculation introduced a NaN value. If the DataFrame is large, we would not be able to observe it. isnull() provides a convenient way to check abnormal values.

In [168]:
missing = aapl_bar.isnull()
print(missing)
print('\n------------------ separate line -----------------\n')
print(missing.describe())

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True
2016-02-29  False  False  False  False        False
2016-03-31  False  False  False  False        False
2016-04-30  False  False  False  False        False
2016-05-31  False  False  False  False        False
2016-06-30  False  False  False  False        False
2016-07-31  False  False  False  False        False
2016-08-31  False  False  False  False        False
2016-09-30  False  False  False  False        False
2016-10-31  False  False  False  False        False
2016-11-30  False  False  False  False        False
2016-12-31  False  False  False  False        False

------------------ separate line -----------------

         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12    

In [169]:
# My example
#print if the respective value is null 
msft_missing = msft_bar.isnull()
print(msft_missing)
print('\n------------------ separate line -----------------\n')
# Gives statistics for each column
print(missing.describe())

             Open  Close  rate_return
Date                                 
2013-01-31  False  False         True
2013-02-28  False  False        False
2013-03-31  False  False        False
2013-04-30  False  False        False
2013-05-31  False  False        False
2013-06-30  False  False        False
2013-07-31  False  False        False
2013-08-31  False  False        False
2013-09-30  False  False        False
2013-10-31  False  False        False
2013-11-30  False  False        False
2013-12-31  False  False        False

------------------ separate line -----------------

         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12     12          11


The row labelled "unique" indicates the number of unique values in each column. Since the "rate_return" column has 2 unique values, it has at least one missing value.

We can deduce the number of missing values by comparing "count" with "freq". There are 12 counts and 11 False values, so there is one True value which corresponds to the missing value.

We can also find the rows with missing values easily:

In [170]:
print(missing[missing.rate_return == True])

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True


In [171]:
# My example
# Only the first row will appear, beacause it's difference with the last (and inexistent) period is NaN 
print(msft_missing[msft_missing.rate_return == True])

             Open  Close  rate_return
Date                                 
2013-01-31  False  False         True


Usually when dealing with missing data, we either delete the whole row or fill it with some value. As we introduced in the Series chapter, the same method **dropna()** and **fillna()** can be applied to a DataFrame.


In [172]:
drop = aapl_bar.dropna()
print(drop)
print('\n---------------------- separate line--------------------\n')
fill = aapl_bar.fillna(0)
print(fill)

              Open      High     Low   Close  rate_return
Date                                                     
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955

---------------------- separate line--------------------

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34     0.000000
2016-02-29   

In [173]:
#My example
#droping NaN values
msft_drop = msft_bar.dropna()
print(msft_drop)
print('\n---------------------- separate line--------------------\n')
# filling NaN values with 0's
msft_fill = msft_bar.fillna(0)
print(msft_fill)

             Open   Close  rate_return
Date                                  
2013-02-28  27.88  27.800         0.09
2013-03-31  28.32  28.605         0.44
2013-04-30  32.56  33.100         4.24
2013-05-31  34.82  34.900         2.26
2013-06-30  34.38  34.545        -0.44
2013-07-31  31.97  31.840        -2.41
2013-08-31  33.37  33.400         1.40
2013-09-30  33.00  33.280        -0.37
2013-10-31  35.66  35.405         2.66
2013-11-30  37.82  38.130         2.16
2013-12-31  37.40  37.410        -0.42

---------------------- separate line--------------------

             Open   Close  rate_return
Date                                  
2013-01-31  27.79  27.450         0.00
2013-02-28  27.88  27.800         0.09
2013-03-31  28.32  28.605         0.44
2013-04-30  32.56  33.100         4.24
2013-05-31  34.82  34.900         2.26
2013-06-30  34.38  34.545        -0.44
2013-07-31  31.97  31.840        -2.41
2013-08-31  33.37  33.400         1.40
2013-09-30  33.00  33.280        -0.37
2013-

####DataFrame Concat
We have seen how to extract a Series from a dataFrame. Now we need to consider how to merge a Series or a DataFrame into another one.

In Pandas, the function concat() allows us to merge multiple Series into a DataFrame:

In [174]:
s1 = pd.Series([143.5, 144.09, 142.73, 144.18, 143.77], name = 'AAPL')
s2 = pd.Series([898.7, 911.71, 906.69, 918.59, 926.99], name = 'GOOG')
data_frame = pd.concat([s1,s2], axis = 1)
print(data_frame)

     AAPL    GOOG
0  143.50  898.70
1  144.09  911.71
2  142.73  906.69
3  144.18  918.59
4  143.77  926.99


In [175]:
# My example
s3 = pd.Series([383.22, 567.5, 386.43, 410.55, 145.49, 448.76, 226.73], name = 'MSFT')
s4 = pd.Series([494.45, 350.81, 435.61, 303.62, 128.92, 339.29, 102.52], name = 'TSLA')
data_frame_2 = pd.concat([s3,s4], axis = 1)
print(data_frame_2)

     MSFT    TSLA
0  383.22  494.45
1  567.50  350.81
2  386.43  435.61
3  410.55  303.62
4  145.49  128.92
5  448.76  339.29
6  226.73  102.52


The "axis = 1" parameter will join two DataFrames by columns:

In [176]:
log_price = np.log(aapl_bar.Close)
log_price.name = 'log_price'
print(log_price)
print('\n---------------------- separate line--------------------\n')
concat = pd.concat([aapl_bar, log_price], axis = 1)
print(concat)

Date
2016-01-31    4.578210
2016-02-29    4.571510
2016-03-31    4.691256
2016-04-30    4.540525
2016-05-31    4.603769
2016-06-30    4.560173
2016-07-31    4.646408
2016-08-31    4.664382
2016-09-30    4.727830
2016-10-31    4.732155
2016-11-30    4.705197
2016-12-31    4.752037
Freq: M, Name: log_price, dtype: float64

---------------------- separate line--------------------

              Open      High     Low   Close  rate_return  log_price
Date                                                                
2016-01-31   94.79   97.3400   94.35   97.34          NaN   4.578210
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678   4.571510
2016-03-31  109.72  109.9000  108.88  108.99     0.127211   4.691256
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921   4.540525
2016-05-31   99.60  100.4000   98.82   99.86     0.065287   4.603769
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660   4.560173
2016-07-31  104.19  104.5500  103.68  104.21     0.090063   4.64640

In [177]:
#My example
#Applying logarithm to every value of the column Open of the table
msft_log_price = np.log(msft_bar.Open)
msft_log_price.name = 'Microsoft_log_price'
print(msft_log_price)
print('\n---------------------- separate line--------------------\n')
# concatenates the column just calculated to the table
msft_concat = pd.concat([msft_bar, msft_log_price], axis = 1)
print(msft_concat)

Date
2013-01-31    3.324676
2013-02-28    3.327910
2013-03-31    3.343568
2013-04-30    3.483085
2013-05-31    3.550192
2013-06-30    3.537475
2013-07-31    3.464798
2013-08-31    3.507657
2013-09-30    3.496508
2013-10-31    3.574030
2013-11-30    3.632838
2013-12-31    3.621671
Freq: M, Name: Microsoft_log_price, dtype: float64

---------------------- separate line--------------------

             Open   Close  rate_return  Microsoft_log_price
Date                                                       
2013-01-31  27.79  27.450          NaN             3.324676
2013-02-28  27.88  27.800         0.09             3.327910
2013-03-31  28.32  28.605         0.44             3.343568
2013-04-30  32.56  33.100         4.24             3.483085
2013-05-31  34.82  34.900         2.26             3.550192
2013-06-30  34.38  34.545        -0.44             3.537475
2013-07-31  31.97  31.840        -2.41             3.464798
2013-08-31  33.37  33.400         1.40             3.507657
2013-09-3

We can also join two DataFrames by rows. Consider these two DataFrames:

In [178]:
df_volume = aapl_table.loc['2016-10':'2017-04',['Volume', 'Split Ratio']].resample('M').agg(lambda x: x[-1])
print(df_volume)
print('\n---------------------- separate line--------------------\n')
df_2017 = aapl_table.loc['2016-10':'2017-04',['Open', 'High', 'Low', 'Close']].resample('M').agg(lambda x: x[-1])
print(df_2017)

                Volume  Split Ratio
Date                               
2016-10-31  26419398.0          1.0
2016-11-30  36162258.0          1.0
2016-12-31  30586265.0          1.0
2017-01-31  49200993.0          1.0
2017-02-28  23482860.0          1.0
2017-03-31  19661651.0          1.0
2017-04-30  20247187.0          1.0

---------------------- separate line--------------------

              Open     High     Low   Close
Date                                       
2016-10-31  113.65  114.230  113.20  113.54
2016-11-30  111.56  112.200  110.27  110.52
2016-12-31  116.65  117.200  115.43  115.82
2017-01-31  121.15  121.390  120.62  121.35
2017-02-28  137.08  137.435  136.70  136.99
2017-03-31  143.72  144.270  143.01  143.66
2017-04-30  144.09  144.300  143.27  143.65


In [179]:
#My example
msft_OpenClose = msft_table.loc['2013-01':'2013-06',['Low', 'Open','Close']].resample('M').mean()
print(msft_OpenClose)
print('\n---------------------- separate line--------------------\n')
msft_OpenClose_2015 = msft_table.loc['2015-01':'2015-06',['Low', 'High', 'Open','Close']].resample('M').mean()
print(msft_OpenClose_2015 )

                  Low       Open      Close
Date                                       
2013-01-31  27.025948  27.243333  27.236714
2013-02-28  27.525526  27.723158  27.703842
2013-03-31  27.933750  28.087500  28.139500
2013-04-30  29.457500  29.642273  29.870000
2013-05-31  33.576955  33.760768  33.950636
2013-06-30  34.319175  34.652500  34.632000

---------------------- separate line--------------------

                  Low       High       Open      Close
Date                                                  
2015-01-31  45.031745  46.070890  45.639000  45.513250
2015-02-28  42.670526  43.272437  42.996842  43.077895
2015-03-31  41.848182  42.527500  42.140000  42.130682
2015-04-30  42.870952  43.736667  43.071905  43.465000
2015-05-31  47.185250  47.976250  47.587005  47.529250
2015-06-30  45.679550  46.465682  46.104091  45.963636


Now we merge the DataFrames with our DataFrame 'aapl_bar'

In [180]:
concat = pd.concat([aapl_bar,df_volume],axis = 1)
print(concat)

              Open      High     Low  ...  rate_return      Volume  Split Ratio
Date                                  ...                                      
2016-01-31   94.79   97.3400   94.35  ...          NaN         NaN          NaN
2016-02-29   96.86   98.2300   96.65  ...    -0.006678         NaN          NaN
2016-03-31  109.72  109.9000  108.88  ...     0.127211         NaN          NaN
2016-04-30   93.99   94.7200   92.51  ...    -0.139921         NaN          NaN
2016-05-31   99.60  100.4000   98.82  ...     0.065287         NaN          NaN
2016-06-30   94.44   95.7700   94.30  ...    -0.042660         NaN          NaN
2016-07-31  104.19  104.5500  103.68  ...     0.090063         NaN          NaN
2016-08-31  105.66  106.5699  105.64  ...     0.018136         NaN          NaN
2016-09-30  112.46  113.3700  111.80  ...     0.065504         NaN          NaN
2016-10-31  113.65  114.2300  113.20  ...     0.004334  26419398.0          1.0
2016-11-30  111.56  112.2000  110.27  ..

In [181]:
#My example
# Merging 2 tables
concat_2 = pd.concat([msft_bar,msft_OpenClose],axis = 1)
print(concat_2)

             Open   Close  rate_return        Low       Open      Close
Date                                                                   
2013-01-31  27.79  27.450          NaN  27.025948  27.243333  27.236714
2013-02-28  27.88  27.800         0.09  27.525526  27.723158  27.703842
2013-03-31  28.32  28.605         0.44  27.933750  28.087500  28.139500
2013-04-30  32.56  33.100         4.24  29.457500  29.642273  29.870000
2013-05-31  34.82  34.900         2.26  33.576955  33.760768  33.950636
2013-06-30  34.38  34.545        -0.44  34.319175  34.652500  34.632000
2013-07-31  31.97  31.840        -2.41        NaN        NaN        NaN
2013-08-31  33.37  33.400         1.40        NaN        NaN        NaN
2013-09-30  33.00  33.280        -0.37        NaN        NaN        NaN
2013-10-31  35.66  35.405         2.66        NaN        NaN        NaN
2013-11-30  37.82  38.130         2.16        NaN        NaN        NaN
2013-12-31  37.40  37.410        -0.42        NaN        NaN    

By default the DataFrame are joined with all of the data. This default options results in zero information loss. We can also merge them by intersection, this is called 'inner join':

In [182]:
concat = pd.concat([aapl_bar,df_volume],axis = 1, join = 'inner')
print(concat)

              Open    High     Low  ...  rate_return      Volume  Split Ratio
Date                                ...                                      
2016-10-31  113.65  114.23  113.20  ...     0.004334  26419398.0          1.0
2016-11-30  111.56  112.20  110.27  ...    -0.026599  36162258.0          1.0
2016-12-31  116.65  117.20  115.43  ...     0.047955  30586265.0          1.0

[3 rows x 7 columns]


In [183]:
# My example
# Inner join
concat_2 = pd.concat([msft_bar,msft_OpenClose],axis = 1, join = 'inner')
print(concat_2)

             Open   Close  rate_return        Low       Open      Close
Date                                                                   
2013-01-31  27.79  27.450          NaN  27.025948  27.243333  27.236714
2013-02-28  27.88  27.800         0.09  27.525526  27.723158  27.703842
2013-03-31  28.32  28.605         0.44  27.933750  28.087500  28.139500
2013-04-30  32.56  33.100         4.24  29.457500  29.642273  29.870000
2013-05-31  34.82  34.900         2.26  33.576955  33.760768  33.950636
2013-06-30  34.38  34.545        -0.44  34.319175  34.652500  34.632000


Only the intersection part was left if use 'inner join' method. Now let's try to append a DataFrame to another one:

In [184]:
append = aapl_bar.append(df_2017)
print(append)

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34          NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955
2016-10-31  113.65  114.2300  113.20  113.54          NaN
2016-11-30  111.56  112.2000  110.27  110.52          NaN
2016-12-31  116.65  117.2000  115.43  115.82          NaN
2017-01-31  12

In [185]:
# My example
append_2 = msft_OpenClose.append(msft_OpenClose_2015)
print(append_2)

                  Low       Open      Close       High
Date                                                  
2013-01-31  27.025948  27.243333  27.236714        NaN
2013-02-28  27.525526  27.723158  27.703842        NaN
2013-03-31  27.933750  28.087500  28.139500        NaN
2013-04-30  29.457500  29.642273  29.870000        NaN
2013-05-31  33.576955  33.760768  33.950636        NaN
2013-06-30  34.319175  34.652500  34.632000        NaN
2015-01-31  45.031745  45.639000  45.513250  46.070890
2015-02-28  42.670526  42.996842  43.077895  43.272437
2015-03-31  41.848182  42.140000  42.130682  42.527500
2015-04-30  42.870952  43.071905  43.465000  43.736667
2015-05-31  47.185250  47.587005  47.529250  47.976250
2015-06-30  45.679550  46.104091  45.963636  46.465682


'Append' is essentially to concat two DataFrames by axis = 0, thus here is an alternative way to append:

In [186]:
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat)

              Open      High     Low   Close  rate_return
Date                                                     
2016-01-31   94.79   97.3400   94.35   97.34          NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678
2016-03-31  109.72  109.9000  108.88  108.99     0.127211
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921
2016-05-31   99.60  100.4000   98.82   99.86     0.065287
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660
2016-07-31  104.19  104.5500  103.68  104.21     0.090063
2016-08-31  105.66  106.5699  105.64  106.10     0.018136
2016-09-30  112.46  113.3700  111.80  113.05     0.065504
2016-10-31  113.65  114.2300  113.20  113.54     0.004334
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599
2016-12-31  116.65  117.2000  115.43  115.82     0.047955
2016-10-31  113.65  114.2300  113.20  113.54          NaN
2016-11-30  111.56  112.2000  110.27  110.52          NaN
2016-12-31  116.65  117.2000  115.43  115.82          NaN
2017-01-31  12

In [187]:
# My example
# concatenating rows
concat_2 = pd.concat([msft_OpenClose, msft_OpenClose_2015], axis = 0)
print(concat_2)

                  Low       Open      Close       High
Date                                                  
2013-01-31  27.025948  27.243333  27.236714        NaN
2013-02-28  27.525526  27.723158  27.703842        NaN
2013-03-31  27.933750  28.087500  28.139500        NaN
2013-04-30  29.457500  29.642273  29.870000        NaN
2013-05-31  33.576955  33.760768  33.950636        NaN
2013-06-30  34.319175  34.652500  34.632000        NaN
2015-01-31  45.031745  45.639000  45.513250  46.070890
2015-02-28  42.670526  42.996842  43.077895  43.272437
2015-03-31  41.848182  42.140000  42.130682  42.527500
2015-04-30  42.870952  43.071905  43.465000  43.736667
2015-05-31  47.185250  47.587005  47.529250  47.976250
2015-06-30  45.679550  46.104091  45.963636  46.465682


Please note that if the two DataFrame have some columns with the same column names, these columns are considered to be the same and will be merged. It's very important to have the right column names. If we change a column names here:

In [188]:
df_2017.columns = ['Change', 'High','Low','Close']
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat)

              Open      High     Low   Close  rate_return  Change
Date                                                             
2016-01-31   94.79   97.3400   94.35   97.34          NaN     NaN
2016-02-29   96.86   98.2300   96.65   96.69    -0.006678     NaN
2016-03-31  109.72  109.9000  108.88  108.99     0.127211     NaN
2016-04-30   93.99   94.7200   92.51   93.74    -0.139921     NaN
2016-05-31   99.60  100.4000   98.82   99.86     0.065287     NaN
2016-06-30   94.44   95.7700   94.30   95.60    -0.042660     NaN
2016-07-31  104.19  104.5500  103.68  104.21     0.090063     NaN
2016-08-31  105.66  106.5699  105.64  106.10     0.018136     NaN
2016-09-30  112.46  113.3700  111.80  113.05     0.065504     NaN
2016-10-31  113.65  114.2300  113.20  113.54     0.004334     NaN
2016-11-30  111.56  112.2000  110.27  110.52    -0.026599     NaN
2016-12-31  116.65  117.2000  115.43  115.82     0.047955     NaN
2016-10-31     NaN  114.2300  113.20  113.54          NaN  113.65
2016-11-30

In [189]:
# My example 
msft_OpenClose.columns = msft_bar.columns
concat = pd.concat([msft_bar, msft_OpenClose], axis = 0)
print(concat)

                 Open      Close  rate_return
Date                                         
2013-01-31  27.790000  27.450000          NaN
2013-02-28  27.880000  27.800000     0.090000
2013-03-31  28.320000  28.605000     0.440000
2013-04-30  32.560000  33.100000     4.240000
2013-05-31  34.820000  34.900000     2.260000
2013-06-30  34.380000  34.545000    -0.440000
2013-07-31  31.970000  31.840000    -2.410000
2013-08-31  33.370000  33.400000     1.400000
2013-09-30  33.000000  33.280000    -0.370000
2013-10-31  35.660000  35.405000     2.660000
2013-11-30  37.820000  38.130000     2.160000
2013-12-31  37.400000  37.410000    -0.420000
2013-01-31  27.025948  27.243333    27.236714
2013-02-28  27.525526  27.723158    27.703842
2013-03-31  27.933750  28.087500    28.139500
2013-04-30  29.457500  29.642273    29.870000
2013-05-31  33.576955  33.760768    33.950636
2013-06-30  34.319175  34.652500    34.632000


Since the column name of 'Open' has been changed, the new DataFrame has an new column named 'Change'.

###Summary
Hereby we introduced the most import part of python: resampling and DataFrame manipulation. We only introduced the most commonly used method in Financial data analysis. There are also many methods used in data mining, which are also beneficial. You can always check the Pandas official documentations for help.